# PyTorch Tensor Introduction: Building Blocks of Deep Learning

Welcome to the comprehensive introduction to **PyTorch tensors** - the fundamental data structure that powers all computations in PyTorch. This tutorial provides a deep dive into tensor creation, manipulation, and operations, specifically designed for learners transitioning from TensorFlow to PyTorch.

## Learning Objectives
By the end of this tutorial, you will master:
1. **Creating Tensors**: Various factory methods and initialization techniques
2. **Tensor Operations**: Arithmetic, broadcasting, and in-place operations
3. **Tensor Shapes**: Manipulating dimensions with unsqueeze, squeeze, and reshape
4. **GPU Acceleration**: Moving tensors to different devices for hardware acceleration
5. **NumPy Bridge**: Seamless interoperability between PyTorch and NumPy

## Key Differences: TensorFlow vs PyTorch Tensors
| Aspect | TensorFlow | PyTorch |
|--------|------------|---------|
| **Creation** | `tf.constant([1, 2, 3])` | `torch.tensor([1, 2, 3])` |
| **Empty Tensors** | `tf.Variable(tf.zeros([2, 3]))` | `torch.empty(2, 3)` |
| **Random** | `tf.random.normal([2, 3])` | `torch.randn(2, 3)` |
| **Device** | Automatic distribution | Explicit `.to(device)` |
| **NumPy** | `.numpy()` method | Direct `.numpy()` sharing memory |
| **Execution** | Graph/Eager modes | Always eager (dynamic) |

---

## 1. Environment Setup and Runtime Detection

Following PyTorch best practices for cross-platform compatibility and device management:

In [ ]:
# Environment Detection and Setup
import sys
import subprocess
import os
import time

# Detect the runtime environment
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules or "kaggle" in os.environ.get('KAGGLE_URL_BASE', '')
IS_LOCAL = not (IS_COLAB or IS_KAGGLE)

print(f"Environment detected:")
print(f"  - Local: {IS_LOCAL}")
print(f"  - Google Colab: {IS_COLAB}")
print(f"  - Kaggle: {IS_KAGGLE}")

# Platform-specific system setup
if IS_COLAB:
    print("\nSetting up Google Colab environment...")
    !apt update -qq
    !apt install -y -qq software-properties-common
elif IS_KAGGLE:
    print("\nSetting up Kaggle environment...")
    # Kaggle usually has most packages pre-installed
else:
    print("\nSetting up local environment...")

# Install required packages for this notebook
required_packages = [
    "torch",
    "torchvision", 
    "numpy",
    "matplotlib"
]

print("\nVerifying required packages...")
for package in required_packages:
    try:
        if package == "torch":
            import torch
            print(f"✓ {package} {torch.__version__}")
        elif package == "torchvision":
            import torchvision
            print(f"✓ {package} {torchvision.__version__}")
        elif package == "numpy":
            import numpy as np
            print(f"✓ {package} {np.__version__}")
        elif package == "matplotlib":
            import matplotlib
            print(f"✓ {package} {matplotlib.__version__}")
    except ImportError:
        print(f"❌ {package} not found")
        if IS_COLAB or IS_KAGGLE:
            !pip install -q {package}
        else:
            subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], 
                          capture_output=True)
        print(f"📦 Installed {package}")

In [ ]:
# Import essential libraries and setup device detection
import torch
import numpy as np
import matplotlib.pyplot as plt
import platform

def detect_device():
    """
    Detect the best available PyTorch device with comprehensive hardware support.
    
    Priority order:
    1. CUDA (NVIDIA GPUs) - Best performance for deep learning
    2. MPS (Apple Silicon) - Optimized for M1/M2/M3 Macs  
    3. CPU (Universal) - Always available fallback
    
    Returns:
        torch.device: The optimal device for PyTorch operations
        str: Human-readable device description for logging
    """
    # Check for CUDA (NVIDIA GPU)
    if torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_name = torch.cuda.get_device_name(0)
        device_info = f"CUDA GPU: {gpu_name}"
        
        cuda_version = torch.version.cuda
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        
        print(f"🚀 Using CUDA acceleration")
        print(f"   GPU: {gpu_name}")
        print(f"   CUDA Version: {cuda_version}")
        print(f"   GPU Memory: {gpu_memory:.1f} GB")
        
        return device, device_info
    
    # Check for MPS (Apple Silicon)
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = torch.device("mps")
        device_info = "Apple Silicon MPS"
        
        system_info = platform.uname()
        
        print(f"🍎 Using Apple Silicon MPS acceleration")
        print(f"   System: {system_info.system} {system_info.release}")
        print(f"   Machine: {system_info.machine}")
        
        return device, device_info
    
    # Fallback to CPU
    else:
        device = torch.device("cpu")
        device_info = "CPU (No GPU acceleration available)"
        
        cpu_count = torch.get_num_threads()
        system_info = platform.uname()
        
        print(f"💻 Using CPU (no GPU acceleration detected)")
        print(f"   Processor: {system_info.processor}")
        print(f"   PyTorch Threads: {cpu_count}")
        print(f"   System: {system_info.system} {system_info.release}")
        
        return device, device_info

# Detect and set up device
device, device_info = detect_device()

print(f"\n✅ PyTorch {torch.__version__} ready!")
print(f"📱 Device selected: {device}")
print(f"📊 Device info: {device_info}")

# Set global device for the notebook
DEVICE = device

## 2. Creating Tensors: Factory Methods and Initialization

PyTorch provides various methods to create tensors. Let's explore the fundamental tensor creation techniques using Australian tourism data examples.

In [ ]:
# 1. Basic tensor creation with torch.empty()
print("📦 1. Basic Tensor Creation with torch.empty()\n")

# torch.empty() allocates memory without initializing values
# Values will be whatever was in memory at the time
empty_tensor = torch.empty(3, 4)
print(f"Empty tensor (3x4):\n{empty_tensor}")
print(f"Shape: {empty_tensor.shape}")
print(f"Data type: {empty_tensor.dtype}")
print(f"Device: {empty_tensor.device}")

print("\n⚠️  Note: torch.empty() values are uninitialized!")
print("   TensorFlow equivalent: tf.Variable(tf.zeros([3, 4])) or tf.empty([3, 4])")
print("   Use torch.empty() when you'll immediately fill the tensor with data")

# Compare with TensorFlow approach
print("\n📊 TensorFlow vs PyTorch Empty Tensors:")
print("   TensorFlow: tf.Variable(tf.zeros([3, 4])) # Usually initialize with zeros")
print("   PyTorch:    torch.empty(3, 4)           # Faster, but uninitialized")

In [ ]:
# 2. Common factory methods for predictable initialization
print("🎯 2. Common Factory Methods: Zeros, Ones, and Random\n")

# Australian tourism data: visitor counts by state (in thousands)
print("Example: Australian state tourism visitor data\n")

# Zeros tensor - useful for initialization
visitor_data = torch.zeros(8, 4)  # 8 states/territories, 4 quarters
print(f"Visitor data initialized (8 states × 4 quarters):\n{visitor_data}")
print(f"Shape: {visitor_data.shape}")

# Ones tensor - useful for masks and default weights
base_tourism_score = torch.ones(8)  # Base score of 1.0 for each state
print(f"\nBase tourism scores: {base_tourism_score}")

# Random tensors - crucial for neural network initialization
print("\n🎲 Random Tensor Creation:")

# Set seed for reproducibility in documentation
torch.manual_seed(42)

# Random numbers from normal distribution (mean=0, std=1)
weather_variations = torch.randn(8, 12)  # 8 states, 12 months
print(f"Weather variations (normal distribution):\n{weather_variations[:3, :6]}...")  # Show subset
print(f"Shape: {weather_variations.shape}")
print(f"Mean: {weather_variations.mean():.4f} (should be ~0)")
print(f"Std: {weather_variations.std():.4f} (should be ~1)")

# Random numbers from uniform distribution [0, 1)
tourism_ratings = torch.rand(8, 5)  # 8 states, 5 categories
print(f"\nTourism ratings (uniform [0,1)):\n{tourism_ratings}")
print(f"Min: {tourism_ratings.min():.4f}, Max: {tourism_ratings.max():.4f}")

print("\n📊 TensorFlow vs PyTorch Random Tensors:")
print("   TensorFlow: tf.random.normal([8, 12])    PyTorch: torch.randn(8, 12)")
print("   TensorFlow: tf.random.uniform([8, 5])    PyTorch: torch.rand(8, 5)")

## 3. Tensor Operations: Arithmetic, Broadcasting, and Transformations

Explore the rich set of operations available for tensor manipulation, from basic arithmetic to advanced broadcasting.

In [ ]:
# 1. Arithmetic operations with scalars
print("➕ 1. Arithmetic Operations with Scalars\n")

# Australian hotel prices per night (AUD)
hotel_prices = torch.tensor([150.0, 200.0, 180.0, 220.0, 160.0], dtype=torch.float32)
cities = ["Sydney", "Melbourne", "Brisbane", "Perth", "Adelaide"]

print(f"Original hotel prices (AUD): {hotel_prices}")
print(f"Cities: {cities}")

# Scalar arithmetic - operations apply element-wise
gst_rate = 0.1  # 10% GST in Australia
discount = 20.0  # $20 discount

# Addition and subtraction
discounted_prices = hotel_prices - discount
print(f"\nAfter $20 discount: {discounted_prices}")

# Multiplication and division
prices_with_gst = hotel_prices * (1 + gst_rate)
print(f"With 10% GST: {prices_with_gst}")

weekly_prices = hotel_prices * 7
print(f"Weekly rates (×7): {weekly_prices}")

# Mathematical functions
log_prices = torch.log(hotel_prices)  # Natural logarithm
rounded_prices = torch.round(prices_with_gst)

print(f"\nLog prices: {log_prices}")
print(f"Rounded GST prices: {rounded_prices}")

print("\n📊 TensorFlow equivalent operations:")
print("   TensorFlow: tf.add(prices, -20)     PyTorch: prices - 20")
print("   TensorFlow: tf.multiply(prices, 1.1) PyTorch: prices * 1.1")
print("   TensorFlow: tf.math.log(prices)     PyTorch: torch.log(prices)")

## 4. Tensor Shapes and Dimensions

Learn how to manipulate tensor shapes using unsqueeze, squeeze, and reshape operations - essential for deep learning.

In [ ]:
# Tensor shape manipulation
print("🔄 Tensor Shape Manipulation: Australian Text Processing\n")

# Simulate tokenized Australian tourism text
# Example: "Sydney Opera House is beautiful" → token IDs
original_tokens = torch.tensor([15, 67, 89, 23, 45, 12, 78, 34, 56, 91, 23, 67])
print(f"Original tokens: {original_tokens}")
print(f"Shape: {original_tokens.shape} (12 tokens)")

# 1. Reshape to different dimensions
print("\n📐 Reshaping Operations:")

# Reshape to 3x4 matrix
reshaped_3x4 = original_tokens.reshape(3, 4)
print(f"Reshaped to 3x4:\n{reshaped_3x4}")

# Reshape to 2x6 matrix
reshaped_2x6 = original_tokens.reshape(2, 6)
print(f"\nReshaped to 2x6:\n{reshaped_2x6}")

# Use -1 for automatic dimension calculation
reshaped_auto = original_tokens.reshape(4, -1)
print(f"\nReshaped to 4x? (auto-calculated):\n{reshaped_auto}")
print(f"Auto shape: {reshaped_auto.shape}")

# 2. Adding dimensions with unsqueeze
print("\n📏 Adding Dimensions (unsqueeze):")

# Add batch dimension (common in deep learning)
with_batch_dim = original_tokens.unsqueeze(0)
print(f"With batch dimension: {with_batch_dim.shape} (1 × 12)")

# Add channel dimension
with_channel_dim = original_tokens.unsqueeze(1)
print(f"With channel dimension: {with_channel_dim.shape} (12 × 1)")

# 3. Removing dimensions with squeeze
print("\n🗜️ Removing Dimensions (squeeze):")

# Remove the batch dimension
squeezed = with_batch_dim.squeeze(0)
print(f"After squeezing batch dim: {squeezed.shape}")

# Practical NLP example: preparing for embedding layer
print("\n💡 Practical NLP Example: Preparing for Embedding Layer")
# Simulate batch of sentences with different lengths (padded)
batch_sentences = torch.tensor([
    [15, 67, 89, 23, 0, 0],    # Sentence 1 (4 real tokens + 2 padding)
    [156, 78, 234, 45, 167, 98], # Sentence 2 (6 real tokens)
    [134, 56, 12, 0, 0, 0]      # Sentence 3 (3 real tokens + 3 padding)
], dtype=torch.long)

print(f"Batch of sentences: {batch_sentences.shape} (batch_size × seq_length)")
print(f"Batch:\n{batch_sentences}")

print("\n📊 TensorFlow vs PyTorch Reshaping:")
print("   TensorFlow: tf.reshape(x, [3, 4])    PyTorch: x.reshape(3, 4) or x.view(3, 4)")
print("   TensorFlow: tf.expand_dims(x, 0)     PyTorch: x.unsqueeze(0)")
print("   TensorFlow: tf.squeeze(x, 0)         PyTorch: x.squeeze(0)")

## 5. GPU Acceleration and Device Management

Learn how to leverage hardware acceleration by moving tensors between devices (CPU, CUDA, MPS).

In [ ]:
# GPU acceleration and device management
print("🚀 GPU Acceleration and Device Management\n")

# Australian city data for device testing
city_data = torch.tensor([5.3, 5.1, 2.6, 2.1, 1.4], dtype=torch.float32)
cities = ["Sydney", "Melbourne", "Brisbane", "Perth", "Adelaide"]

print(f"Original data: {city_data}")
print(f"Device: {city_data.device}")
print(f"Data type: {city_data.dtype}")

# Check available devices
print(f"\n🔍 Device Availability:")
print(f"CUDA available: {torch.cuda.is_available()}")
if hasattr(torch.backends, 'mps'):
    print(f"MPS available: {torch.backends.mps.is_available()}")
else:
    print(f"MPS available: False (not supported in this PyTorch version)")

# Move to different devices
print(f"\n📱 Moving Tensors Between Devices:")

# Explicit CPU placement
cpu_data = city_data.to('cpu')
print(f"CPU data: {cpu_data.device}")

# Try CUDA if available
if torch.cuda.is_available():
    print("\n🚀 CUDA GPU available - demonstrating GPU operations:")
    gpu_data = city_data.to('cuda')
    print(f"GPU data device: {gpu_data.device}")
    
    # Perform computation on GPU
    gpu_result = gpu_data * 2 + 1
    print(f"GPU computation result: {gpu_result}")
    
    # Move back to CPU for display
    cpu_result = gpu_result.cpu()
    print(f"Result moved to CPU: {cpu_result}")
    
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    print("\n🍎 MPS (Apple Silicon) available - demonstrating MPS operations:")
    mps_data = city_data.to('mps')
    print(f"MPS data device: {mps_data.device}")
    
    # Perform computation on MPS
    mps_result = mps_data * 2 + 1
    print(f"MPS computation result: {mps_result}")
    
    # Move back to CPU for display
    cpu_result = mps_result.cpu()
    print(f"Result moved to CPU: {cpu_result}")
    
else:
    print("\n💻 No GPU acceleration available - using CPU only")
    print("This is normal for most development environments")
    
# Device-aware tensor creation
print(f"\n🎯 Device-Aware Tensor Creation:")
device_aware_tensor = torch.randn(3, 4, device=DEVICE)
print(f"Created on {DEVICE}: {device_aware_tensor.device}")

print("\n💡 Best Practices for Device Management:")
print("   • Always check device availability before using")
print("   • Move models and data to the same device")
print("   • Use .to(device) for explicit device placement")
print("   • Consider memory limitations when using GPU")

print("\n📊 TensorFlow vs PyTorch Device Management:")
print("   TensorFlow: with tf.device('/GPU:0'): ...")
print("   PyTorch:    tensor.to('cuda')")

## 6. PyTorch/NumPy Bridge: Seamless Interoperability

Discover the powerful integration between PyTorch tensors and NumPy arrays, including memory sharing and conversion methods.

In [ ]:
# PyTorch/NumPy bridge
print("🌉 PyTorch/NumPy Bridge: Seamless Interoperability\n")

# Australian tourism statistics using NumPy arrays
print("Example: Converting between PyTorch and NumPy for data analysis\n")

# Start with NumPy array (common in data science)
import numpy as np

# Australian state populations (millions)
state_populations = np.array([8.2, 6.7, 5.2, 2.7, 1.8, 0.5, 0.6, 0.4], dtype=np.float32)
states = ["NSW", "VIC", "QLD", "WA", "SA", "TAS", "ACT", "NT"]

print(f"NumPy array: {state_populations}")
print(f"NumPy dtype: {state_populations.dtype}")
print(f"NumPy shape: {state_populations.shape}")

# 1. Convert NumPy to PyTorch with torch.from_numpy()
print("\n📥 NumPy → PyTorch Conversion:")

pytorch_populations = torch.from_numpy(state_populations)
print(f"PyTorch tensor: {pytorch_populations}")
print(f"PyTorch dtype: {pytorch_populations.dtype}")
print(f"PyTorch shape: {pytorch_populations.shape}")

# Check memory sharing
print(f"\n🧠 Memory Sharing Check:")
print(f"Shares memory: {pytorch_populations.data_ptr() == state_populations.__array_interface__['data'][0]}")
print("Note: from_numpy() creates a tensor that shares memory with the NumPy array")

# Demonstrate shared memory
original_value = state_populations[0]
print(f"\nBefore modification - NumPy[0]: {state_populations[0]}, Tensor[0]: {pytorch_populations[0]}")
state_populations[0] = 999  # Modify NumPy array
print(f"After modifying NumPy - NumPy[0]: {state_populations[0]}, Tensor[0]: {pytorch_populations[0]}")
state_populations[0] = original_value  # Restore original value

# 2. Convert PyTorch to NumPy with .numpy()
print("\n📤 PyTorch → NumPy Conversion:")

# Create PyTorch tensor with tourism data
tourism_scores = torch.tensor([4.8, 4.6, 4.7, 4.3, 4.2, 4.1, 3.9, 4.0], dtype=torch.float32)
print(f"Tourism scores (PyTorch): {tourism_scores}")

# Convert to NumPy
tourism_numpy = tourism_scores.numpy()
print(f"Tourism scores (NumPy): {tourism_numpy}")
print(f"NumPy dtype: {tourism_numpy.dtype}")

# 3. Device considerations
print("\n🔧 Device Considerations: CPU vs GPU Tensors")

# Create tensor on CPU
cpu_tensor = torch.tensor([1.0, 2.0, 3.0, 4.0], device='cpu')
print(f"CPU tensor: {cpu_tensor}")
print(f"Device: {cpu_tensor.device}")

# Convert CPU tensor to NumPy (works directly)
cpu_numpy = cpu_tensor.numpy()
print(f"CPU → NumPy: {cpu_numpy}")

# Device handling demonstration
if torch.cuda.is_available():
    print("\n🚀 CUDA available - GPU tensor demonstration:")
    gpu_tensor = cpu_tensor.to('cuda')
    print(f"GPU tensor: {gpu_tensor}")
    print(f"Device: {gpu_tensor.device}")
    
    # Must move to CPU before converting to NumPy
    gpu_to_numpy = gpu_tensor.cpu().numpy()
    print(f"GPU → CPU → NumPy: {gpu_to_numpy}")
    
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    print("\n🍎 MPS available - Apple Silicon demonstration:")
    mps_tensor = cpu_tensor.to('mps')
    print(f"MPS tensor: {mps_tensor}")
    print(f"Device: {mps_tensor.device}")
    
    # Must move to CPU before converting to NumPy
    mps_to_numpy = mps_tensor.cpu().numpy()
    print(f"MPS → CPU → NumPy: {mps_to_numpy}")
    
else:
    print("\n💻 No GPU/MPS acceleration available - using CPU only")

print("\n⚠️ Important Notes:")
print("- NumPy arrays are always on CPU")
print("- GPU/MPS tensors must be moved to CPU before .numpy()")
print("- from_numpy() always creates CPU tensors")
print("- Use .to(device) to move tensors between devices")

print("\n📊 TensorFlow vs PyTorch NumPy Integration:")
print("   TensorFlow: tf.convert_to_tensor(numpy_array)")
print("   PyTorch:    torch.from_numpy(numpy_array)")
print("   TensorFlow: tensor.numpy()")
print("   PyTorch:    tensor.numpy()")

## Summary: PyTorch Tensor Fundamentals

Congratulations! You've completed the comprehensive introduction to PyTorch tensors. Let's review what you've learned:

### 🎯 Key Concepts Mastered

1. **Tensor Creation**
   - `torch.empty()` for uninitialized tensors
   - `torch.zeros()`, `torch.ones()` for initialized tensors
   - `torch.randn()`, `torch.rand()` for random tensors
   - `torch.manual_seed()` for reproducibility
   - "Like" methods (`zeros_like()`, `ones_like()`, etc.)
   - Creating from Python collections with `torch.tensor()`

2. **Tensor Operations**
   - Element-wise arithmetic with scalars and tensors
   - Broadcasting for operations on different-shaped tensors
   - In-place operations with `_` suffix (e.g., `add_()`, `mul_()`)
   - Tensor copying with `clone()` and `detach()`

3. **Shape Manipulation**
   - `reshape()` and `view()` for changing tensor dimensions
   - `unsqueeze()` for adding dimensions
   - `squeeze()` for removing size-1 dimensions

4. **Device Management**
   - Device detection (CUDA, MPS, CPU)
   - Moving tensors with `.to(device)`
   - Device-aware tensor creation

5. **NumPy Integration**
   - `torch.from_numpy()` for NumPy → PyTorch conversion
   - `.numpy()` for PyTorch → NumPy conversion
   - Memory sharing considerations
   - Device compatibility with NumPy

### 🚀 Next Steps

Now that you understand PyTorch tensors, you're ready to:
- Build neural networks with `torch.nn`
- Implement training loops with automatic differentiation
- Work with real datasets using `torch.utils.data`
- Explore advanced tensor operations for NLP and computer vision

### 📚 Additional Resources

- [PyTorch Tensor Documentation](https://pytorch.org/docs/stable/tensors.html)
- [PyTorch Tutorials](https://pytorch.org/tutorials/)
- [Australian Tourism Dataset Examples](https://github.com/vuhung16au/pytorch-mastery)

### 🌟 Key Takeaway

PyTorch tensors are the foundation of all deep learning operations. Their dynamic nature, device flexibility, and seamless NumPy integration make them powerful tools for research and production. The Australian-themed examples demonstrate how these concepts apply to real-world data processing scenarios.

**Happy tensor computing! 🎉**